In [1]:
#import keepsake
import pandas as pd
import numpy as np

from validation_utils import transform_data_for_inference, line_color, downsample_df, run_inference, calculate_error, filter_df_by_signals, get_move_from_path

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

plt.style.use('ggplot')

%matplotlib inline

In [1]:
# define model imports based on experiment ids

from models.optimized.base.random_forest import random_forest_20_acc_x_acc_y_acc_z_gyro_x_gyro_y_4 as rf_1 # 20hz
from models.optimized.base.random_forest import random_forest_50_acc_x_acc_y_acc_z_gyro_x_gyro_y_6 as rf_2 # 50hz

from models.optimized.base.decision_tree import decision_tree_25hz_acc_x_acc_y_acc_z_gyro_x as dt_1 # 25hz
from models.optimized.base.decision_tree import decision_tree_25hz_acc_x_acc_y_acc_z_gyro_y as dt_2 # 25hz

In [2]:
# load baseline models for comparison between optimized and baseline (all signals)

from models.baseline.base.random_forest import random_forest_20hz
from models.baseline.base.random_forest import random_forest_50hz

from models.baseline.base.decision_tree import decision_tree_25hz

In [4]:
from importlib.machinery import SourceFileLoader
import importlib.util
import sys
import os




In [6]:
import os

def clean_model_files(models_dir="models/optimized/base"):
    for root, dirs, files in os.walk(models_dir):
        for file in files:
            if file.endswith(".py"):
                file_path = os.path.join(root, file)
                with open(file_path, 'rb') as f:
                    content = f.read()
                cleaned_content = content.replace(b'\x00', b'')
                with open(file_path, 'wb') as f:
                    f.write(cleaned_content)
                print(f"Cleaned: {file_path}")

clean_model_files()


Cleaned: models/optimized/base\decision_tree\decision_tree_25hz_acc_x_acc_y_acc_z_gyro_x.py
Cleaned: models/optimized/base\decision_tree\decision_tree_25hz_acc_x_acc_y_acc_z_gyro_y.py
Cleaned: models/optimized/base\decision_tree\decision_tree_25hz_acc_x_acc_y_acc_z_gyro_z.py
Cleaned: models/optimized/base\decision_tree\decision_tree_decision_tree_25hz.py
Cleaned: models/optimized/base\random_forest\random_forest_20_acc_x_acc_y_acc_z_gyro_x_gyro_y_4.py
Cleaned: models/optimized/base\random_forest\random_forest_20_acc_x_acc_y_acc_z_gyro_x_gyro_y_5.py
Cleaned: models/optimized/base\random_forest\random_forest_20_acc_x_acc_y_acc_z_gyro_x_gyro_y_6.py
Cleaned: models/optimized/base\random_forest\random_forest_20_acc_x_acc_y_acc_z_gyro_x_gyro_z_4.py
Cleaned: models/optimized/base\random_forest\random_forest_25_acc_x_acc_y_acc_z_gyro_x_gyro_y_gyro_z_5.py
Cleaned: models/optimized/base\random_forest\random_forest_25_acc_x_acc_y_acc_z_gyro_x_gyro_y_gyro_z_6.py
Cleaned: models/optimized/base\rand

In [7]:
def save_model_optimized(model, stage, dataset, model_type, exp_id):
    model_dir = f"models/{stage}/{dataset}/{model_type}/"
    os.makedirs(model_dir, exist_ok=True)
    model_filename = f"{model_dir}/{exp_id}.py"
    
    # Generate code and remove null bytes
    py_code = m2cgen.export_to_python(model)
    py_code = py_code.replace('\x00', '')  # Proactively remove null bytes
    
    with open(model_filename, 'w', encoding='utf-8') as f:
        f.write(py_code)
    
    return model_filename


In [8]:
from importlib.machinery import SourceFileLoader
import importlib.util
import sys
import os

def load_py_model(model_path):
    try:
        module_name = os.path.splitext(os.path.basename(model_path))[0]
        loader = SourceFileLoader(module_name, model_path)
        spec = importlib.util.spec_from_loader(module_name, loader)
        
        if not spec:
            raise ValueError(f"Invalid model file: {model_path}")
            
        module = importlib.util.module_from_spec(spec)
        sys.modules[module_name] = module
        spec.loader.exec_module(module)
        return module.score
        
    except ValueError as e:
        print(f"Critical error in {model_path}: {str(e)}")
        raise
